# Parte VI: Proyectos adicionales

Autores: Jordi Bolibar & Facundo Sapienza  
Traducción: Eduardo Villavicencio

En el cuaderno 5, recalculamos un conjunto de datos distribuido que nos permite trabajar con datos en rejilla 2D en lugar de datos a escala de glaciar. En este cuaderno, presentaremos dos proyectos adicionales que pueden ayudarte a explotar estos conjuntos de datos aún más.

Para estos dos proyectos, utilizaremos las siguientes bibliotecas:

In [1]:
## Auxiliary libraries
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
import salem
from pathlib import Path
import os

# OGGM
import oggm.cfg as cfg
from oggm import utils, workflow, tasks, graphics

cfg.initialize(logging_level='WARNING') # initialize OGGM
cfg.PARAMS['border'] = 10
cfg.PARAMS['use_multiprocessing'] = True 

2023-04-20 08:46:47: oggm.cfg: Reading default parameters from the OGGM `params.cfg` configuration file.
2023-04-20 08:46:47: oggm.cfg: Multiprocessing switched OFF according to the parameter file.
2023-04-20 08:46:47: oggm.cfg: Multiprocessing: using all available processors (N=4)
2023-04-20 08:46:49: oggm.cfg: PARAMS['border'] changed from `80` to `10`.
2023-04-20 08:46:49: oggm.cfg: Multiprocessing switched ON after user settings.


## 6.1. Proyecto adicional No. 1: Balance de masa glaciar distribuido

<img src="Figures/eye_logo.png" width="75"/>

En este proyecto, proponemos ir más allá del modelado del balance de masa que hicimos en los cuadernos anteriores, y intentar aprender la distribución espacial del balance de masa glaciar directamente a partir de los datos de balance de masa geodésico en rejilla de Hugonnet et al. (2021). En lugar de intentar modelar un único punto de datos por glaciar, simularemos la distribución espacial del balance de masa (es decir, su gradiente altitudinal).

## 6.1.1 Construcción de un conjunto de datos distribuido

En los cuadernos anteriores trabajamos con datos tabulares. Como puedes ver en la gráfica anterior, ahora trabajaremos con datos distribuidos (2D). Por lo tanto, necesitamos reorganizar y ajustar nuestro conjunto de datos para este nuevo problema.

In [8]:
df_subset = pd.read_csv('Data/df_distributed_subset.csv')#Data/df_distributed_training.csv
df_subset.columns

Index(['Unnamed: 0.1', 'Unnamed: 0', 'RGI_ID', 'PDD_2D', 'rain_2D', 'snow_2D',
       'topo', 'aspect', 'slope', 'dis_from_border', 'glacier_mask',
       'millan_ice_thickness', 'hugonnet_dhdt'],
      dtype='object')

Limpiamos un poco el `DataFrame`.

In [11]:
df_subset = df_subset.drop(['Unnamed: 0.1', 'Unnamed: 0'], axis=1)# Eliminar las columnas innecesarias 

Este es un pequeño subconjunto del conjunto de datos completo, una versión liviana para comenzar a trabajar.

In [12]:
df_subset

,RGI_ID,PDD_2D,rain_2D,snow_2D,topo,aspect,slope,dis_from_border,glacier_mask,millan_ice_thickness,hugonnet_dhdt
0,RGI60-08.00005,609.424791,24237.964135,16140.702605,1024.7790,5.331003,0.102807,1997.02900,0,0.0,0.047467
1,RGI60-08.00005,615.582145,24237.964135,16140.702605,1034.9397,5.075106,0.112442,1955.76070,0,0.0,0.091581
2,RGI60-08.00005,626.451758,24237.964135,16049.763045,1052.8350,4.781625,0.125466,1916.53980,0,0.0,0.083521
3,RGI60-08.00005,632.948060,24237.964135,16049.763045,1063.4498,4.490769,0.125620,1879.49460,0,0.0,0.034988
4,RGI60-08.00005,637.226233,24237.964135,16049.763045,1070.4403,4.222532,0.151456,1844.75610,0,0.0,0.034173
...,...,...,...,...,...,...,...,...,...,...,...
53565,RGI60-08.00014,1053.637775,23169.686477,11568.195539,1352.4573,1.996007,0.119973,357.61713,0,0.0,-0.010636
53566,RGI60-08.00014,1051.288168,23169.686477,11721.653822,1349.4329,2.012404,0.114451,373.89438,0,0.0,-0.016246
53567,RGI60-08.00014,1049.039072,23169.686477,11721.653822,1346.5271,2.080615,0.099085,390.62260,0,0.0,-0.022051
53568,RGI60-08.00014,1046.852415,23169.686477,11721.653822,1343.7019,2.261690,0.075171,407.74625,0,0.0,-0.024491


### CONTINUARÁ

Veremos cómo construir matrices de entrenamiento con todos los datos de balance de masa geodésico, y cómo reestructurar el conjunto de datos de entrenamiento de los cuadernos anteriores para darle la misma forma que las matrices aplanadas de este conjunto de datos. Para muchos datos (por ejemplo, climáticos) que solo están disponibles a baja resolución, podemos simplemente repetir el mismo valor para las precipitaciones y realizar una reducción aproximada para la temperatura.

## 6.2. Proyecto adicional No. 2: Inferencia del grosor del hielo glacia

<img src="Figures/eye_logo.png" width="75"/>

En este proyecto, proponemos simular algo completamente diferente a lo que hemos estado haciendo en las diapositivas anteriores. En lugar de trabajar en el balance de masa glaciar, intentaremos inferir el grosor del hielo glaciar desde una perspectiva basada en datos. OGGM proporciona acceso a muchos conjuntos de datos de glaciares diferentes, incluido Glathida, el conjunto de datos global de observaciones del grosor del hielo glaciar. Estas observaciones cubren un número reducido de glaciares (aproximadamente 2700), y solo de manera parcial en términos de superficie glaciar. No obstante, todos los puntos de datos disponibles se pueden utilizar para entrenar un modelo de aprendizaje automático que infiera el grosor del hielo glaciar a partir de diferentes características de entrada de interés.

Estas diferentes características de entrada podrían ser:
- Elevación de la superficie del glaciar
- Pendiente de la superficie
- Velocidades de la superficie del hielo

In [ ]:
parent_path = os.path.dirname(Path().resolve())
workspace_path = os.path.join(parent_path, 'OGGM_data_Finse_glathida')
#workspace_path = '/home/jovyan/shared/glacier-ml-2022/Mass_Balance_ML_Modelling/Data'

if not os.path.exists(workspace_path):
    os.mkdir(workspace_path)

cfg.PATHS['working_dir'] = workspace_path

In [ ]:
gtd_file = utils.file_downloader('https://cluster.klima.uni-bremen.de/~oggm/glathida/glathida-v3.1.0/data/TTT_per_rgi_id.h5')

glathida = pd.HDFStore(gtd_file)
rgi_ids = glathida.keys()
rgi_ids = [id[1:] for id in rgi_ids]

In [ ]:
# We use the directories with the shop data in it: "W5E5_w_data"
base_url = 'https://cluster.klima.uni-bremen.de/~oggm/gdirs/oggm_v1.6/L3-L5_files/2023.1/elev_bands/W5E5_w_data/'
gdirs = workflow.init_glacier_directories(rgi_ids, from_prepro_level=3, prepro_base_url=base_url, prepro_border=10)

In [ ]:
df = pd.read_hdf(gtd_file, key=rgi_id)
df.plot(y='POINT_LAT', x='POINT_LON', c='THICKNESS', kind='scatter', cmap='viridis');